In [1]:
import os
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor
import torch
import gc

/home/elboardy/.local/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/elboardy/.local/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/elboardy/.local/lib/python3.9/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the A

In [2]:
model_id = "meta-llama/Llama-3.2-90B-Vision-Instruct"
cases_dir = '/media/RLAB-Disk01/(final)merged_images_with_labels_order_and_folders_mask_normalized/'
cache_dir = '/media/RLAB-Disk01/Large-Language-Models-Weights'
offload_folder = '/media/RLAB-Disk01/Large-Language-Models-Weights'

In [3]:
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
#torch.cuda.set_per_process_memory_fraction(0.8)

In [4]:
cases_to_process = []
for case in os.listdir(cases_dir):
    case_dir = os.path.join(cases_dir, case)
    if os.path.isdir(case_dir):
        response_path = os.path.join(case_dir, 'llama-90B.txt')
        if not os.path.exists(response_path):
            cases_to_process.append(case)
print(f"Found {len(cases_to_process)} cases to process.")

Found 40 cases to process.


In [ ]:

model_id = model_id
model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    cache_dir=cache_dir,
    offload_folder=offload_folder,
    attn_implementation="eager",
    use_safetensors=True,
    offload_state_dict=True, 
)
processor = AutoProcessor.from_pretrained(model_id,use_fast=True)


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/37 [00:00<?, ?it/s]

In [6]:
def load_image(image_path):
    try:
        img = Image.open(image_path)
        return img
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None

In [7]:
print("Processing cases...--->")
print(cases_to_process)

Processing cases...--->
['RHUH-0019', 'RHUH-0001', 'RHUH-0002', 'RHUH-0003', 'RHUH-0004', 'RHUH-0005', 'RHUH-0006', 'RHUH-0007', 'RHUH-0008', 'RHUH-0009', 'RHUH-0010', 'RHUH-0011', 'RHUH-0012', 'RHUH-0013', 'RHUH-0014', 'RHUH-0015', 'RHUH-0016', 'RHUH-0017', 'RHUH-0018', 'RHUH-0020', 'RHUH-0021', 'RHUH-0022', 'RHUH-0023', 'RHUH-0024', 'RHUH-0025', 'RHUH-0026', 'RHUH-0027', 'RHUH-0028', 'RHUH-0029', 'RHUH-0030', 'RHUH-0031', 'RHUH-0032', 'RHUH-0033', 'RHUH-0034', 'RHUH-0035', 'RHUH-0036', 'RHUH-0037', 'RHUH-0038', 'RHUH-0039', 'RHUH-0040']


In [ ]:

failed_cases = []

for case in cases_to_process:
    case_dir = os.path.join(cases_dir, case)
    image_files = [
        f for f in os.listdir(case_dir)
        if f.lower().endswith('.png')
    ]

    clinical_information_path = os.path.join(case_dir, 'diagnostic_prompt.txt')
    if not os.path.exists(clinical_information_path):
        print(f"Missing clinical information file for case: {case}")
        continue

    clinical_information = open(clinical_information_path).read()

    system_prompt = """Consider that you are a professional radiologist with several years of experience and you are now treating a patient. Write a fully detailed diagnosis report for this case, avoiding any potential hallucination and paying close attention to all of the batch images attached to this message.

Use the following structure for the report:

## Radiologist's Report

### Patient Information:
- *Age:* 65
- *Sex:* Male
- *Days from earliest imaging to surgery:* 1
- *Histopathological Subtype:* Glioblastoma
- *WHO Grade:* 4
- *IDH Status:* Mutant
- *Preoperative KPS:* 80
- *Preoperative Contrast-Enhancing Tumor Volume (cm³):* 103.21
- *Preoperative T2/FLAIR Abnormality (cm³):* 36.29
- *Extent of Resection (EOR %):* 100.0
- *EOR Type:* Gross Total Resection (GTR)
- *Adjuvant Therapy:* Radiotherapy (RT) + Temozolomide (TMZ)
- *Progression-Free Survival (PFS) Days:* 649
- *Overall Survival (OS) Days:* 736

#### Tumor Characteristics:

#### Segmentation Analysis:

#### Surgical Considerations:

### Clinical Summary:

### Recommendations:

### Prognostic Considerations:

### Follow-Up Plan:

### Additional Notes*(if any)*:

Ensure all findings from all of the images and clinical data provided. Please mention at the end of the report how many images were reviewed."""

    user_prompt = f"""You will be given batches of images, which are different sequences of MRI scans. 
    The images are for patients who are likely to have a brain tumor. Each image will contain up to 10 slices for 5 different sequences and the segmentation masks for the tumor at the bottom row of the image. 
    Additional clinical data about the patient is: 
    {clinical_information}"""

    # Process all images
    for image_file in image_files:
        image_path = os.path.join(case_dir, image_file)
        img = load_image(image_path)
   

    # Prepare the messages for LLaMA Vision
    messages = [
         {"role": "system_admin", "content": [
            {"type": "text", "text": system_prompt},
        ]},
        {"role": "user", "content": [
            {"type": "image"},
            {"type": "text", "text": user_prompt}
        ]}
    ]

    print(f"Messages prepared for case {case}")

    try:


        # Apply the chat template
        input_text = processor.apply_chat_template(messages, add_generation_prompt=True)

    # Process the input and images
        inputs = processor(
        images=img,
        text=input_text,
        add_special_tokens=False,
        return_tensors="pt"
        ).to(model.device)

    # Generate the response
        output = model.generate(**inputs, max_new_tokens=4096)
        response_text = processor.decode(output[0], skip_special_tokens=True)
        
        print(f"Response generated for case {case}, length: {len(response_text)} characters")

    # Save the response
        response_path = os.path.join(case_dir, 'llama-90B.txt')
        with open(response_path, 'w', encoding='utf-8') as f:
            f.write(response_text)

        print(f"Response saved for case {case}.")
        # Memory management
        del inputs, generated_ids, response_text
        gc.collect()
        torch.cuda.empty_cache()
        print(f"Memory cleared for case {case}")

    except torch.cuda.OutOfMemoryError:
        print(f"CUDA out of memory error for case {case}. Skipping this case.")
        failed_cases.append(case)
        continue
    except KeyboardInterrupt:
        print("Interrupted by user. Proceeding to the next case.")
        continue
    except Exception as e:
        print(f"Error processing case {case}: {e}")
        failed_cases.append(case)
        continue

# After processing all cases, save failed cases
failed_cases_path = os.path.join(cases_dir, 'failed_cases_llama_90b.txt')
with open(failed_cases_path, 'w', encoding='utf-8') as f:
    for failed_case in failed_cases:
        f.write(f"{failed_case}\n")
print(f"Failed cases logged in {failed_cases_path}")

Messages prepared for case RHUH-0019
